In [0]:
from pyspark.sql.functions import *

In [0]:
confluentBootstrapServers = 'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092'
confluentApiKey = 'Z2OSAATSA55GQVZK'
confluentSecret = 'a1QC+319eHmIkhcb665BX6v2PCI5utizHlkPmZJQX43Tbx0ZtlWEht4888R1IJqq'
confluentTopicName = 'retail-data-new'
confluentTargetTopicName = 'processed_orders'

In [0]:
orders_df = spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers",confluentBootstrapServers) \
.option("kafka.security.protocol","SASL_SSL") \
.option("kafka.sasl.mechanism","PLAIN") \
.option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.option("subscribe",confluentTopicName) \
.option("startingTimestamp",1) \
.option("maxOffsetsPerTrigger",50) \
.load()


In [0]:
converted_orders_df = orders_df.selectExpr("CAST(key as string) AS key","CAST(value as string) AS value","topic","partition","offset","timestamp","timestampType")

In [0]:
orders_schema = "order_id long,customer_id long,customer_fname string,customer_lname string,city string,state string,pincode long,line_items array<struct<order_item_id: long,order_item_product_id: long,order_item_quantity: long,order_item_product_price: float,order_item_subtotal: float>>"

In [0]:
parsed_orders_df = converted_orders_df.select("key", from_json("value",orders_schema).alias("value"),"topic","partition","offset","timestamp","timestampType")

In [0]:
parsed_orders_df.createOrReplaceTempView("orders")

In [0]:
filtered_orders = spark.sql("select CAST(key as string) AS key, CAST(value as string) AS value from orders where value.city = 'Chicago'")

In [0]:
query = filtered_orders \
    .writeStream \
    .queryName("ingestionquery") \
    .format("kafka") \
    .outputMode("append") \
    .option("checkpointLocation", "checkpointdir304") \
    .option("kafka.bootstrap.servers", confluentBootstrapServers) \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
    .option("kafka.ssl.endpoint.identification.algorithm", "https") \
    .option("topic", confluentTargetTopicName) \
    .option("kafka.batch.size", "16384") \
    .option("kafka.linger.ms", "1") \
    .trigger(processingTime='2 seconds') \
    .start()

Out[22]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f08ae45f220>

In [0]:
try:
    query.awaitTermination()
except Exception as e:
    print(f"Streaming query failed: {str(e)}")
    query.stop()